In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import nept

from matplotlib import animation
from IPython.display import HTML

from startup import extract_xy, sort_led_locations, correct_targets, median_filter, remove_jumps_to_feeder
from loading_data import get_data, unzip_nvt_file, zip_nvt_file
from analyze_decode_bytrial import get_trials

In [ ]:
thisdir = os.getcwd()
dataloc = os.path.join(thisdir, 'cache', 'data')
pickle_filepath = os.path.join(thisdir, "cache", "pickled")
output_filepath = os.path.join(thisdir, "plots", "correcting_position")

In [ ]:
import info.r068d6 as info

In [ ]:
events, position, spikes, lfp, lfp_theta = get_data(info)

In [ ]:
plt.plot(position.x, position.y, "b.")
plt.show()

In [ ]:
xedges, yedges = nept.get_xyedges(position)

trial_epochs = get_trials(events, info.task_times["phase3"])
trial_idx = 18
start = trial_epochs[trial_idx].start
stop = trial_epochs[trial_idx].stop

In [ ]:
trial_position = position.time_slice(start, stop)

In [ ]:
plt.plot(trial_position.x, trial_position.y, "g.")
plt.show()

In [ ]:
fig, ax = plt.subplots()

xx, yy = np.meshgrid(xedges, yedges)

pad_amount = 5
ax.set_xlim((np.floor(np.min(trial_position.x))-pad_amount, np.ceil(np.max(trial_position.x))+pad_amount))
ax.set_ylim((np.floor(np.min(trial_position.y))-pad_amount, np.ceil(np.max(trial_position.y))+pad_amount))

plt.plot(trial_position.x, trial_position.y, ".", color="#bdbdbd")

n_timebins = trial_position.n_samples
rat_position, = ax.plot([], [], "<", color="r")

fig.tight_layout()


def init():
    rat_position.set_data([], [])
    return rat_position


def animate(i):
    rat_position.set_data(trial_position.x[i], trial_position.y[i])
    return rat_position

anim = animation.FuncAnimation(fig, animate, frames=n_timebins, interval=80, 
                               blit=False, repeat=True)

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
# unzip_nvt_file(dataloc, vt_filename)

In [ ]:
# os.path.join(dataloc, vt_filename)

In [ ]:
# vt_filename = info.position_filename[:-4]

In [ ]:
# import zipfile
# datapath=dataloc
# filename=vt_filename

# with zipfile.ZipFile(os.path.join(datapath, filename+'.zip'), 'r') as file:
#         file.extractall(datapath)

# # file.close()

In [ ]:
# os.path.join(datapath, filename+'.zip')

In [ ]:
position_filename = os.path.join(dataloc, info.position_filename)
nvt_position = nept.load_nvt(position_filename)

In [ ]:
raw_position = nept.Position(np.hstack(np.array([nvt_position['x'], nvt_position['y']])[..., np.newaxis]), 
                             nvt_position['time'])

In [ ]:
sliced_raw_position = raw_position.time_slice(start, stop)

In [ ]:
plt.plot(sliced_raw_position.x, sliced_raw_position.y, 'k.')
plt.show()

In [ ]:
position = sliced_raw_position
fig, ax = plt.subplots()

xx, yy = np.meshgrid(xedges, yedges)

pad_amount = 5
ax.set_xlim((np.floor(np.min(position.x))-pad_amount, np.ceil(np.max(position.x))+pad_amount))
ax.set_ylim((np.floor(np.min(position.y))-pad_amount, np.ceil(np.max(position.y))+pad_amount))

plt.plot(position.x, position.y, ".", color="#bdbdbd")

n_timebins = position.n_samples
rat_position, = ax.plot([], [], "<", color="r")

fig.tight_layout()


def init():
    rat_position.set_data([], [])
    return rat_position


def animate(i):
    rat_position.set_data(position.x[i], position.y[i])
    return rat_position

raw_anim = animation.FuncAnimation(fig, animate, frames=n_timebins, interval=80, 
                                   blit=False, repeat=True)

In [ ]:
HTML(raw_anim.to_html5_video())

load_shortcut_position in emi_shortcut goes through the following sequence to clean up the position data

In [ ]:
def animate_trial(x, y, time, start, stop):
    
    position = nept.Position(np.hstack(np.array([x, y])[..., np.newaxis]), 
                             time)
    
    position = position.time_slice(start, stop)
    
    fig, ax = plt.subplots()

    xx, yy = np.meshgrid(xedges, yedges)

    pad_amount = 5
    ax.set_xlim((np.floor(np.min(position.x))-pad_amount, np.ceil(np.max(position.x))+pad_amount))
    ax.set_ylim((np.floor(np.min(position.y))-pad_amount, np.ceil(np.max(position.y))+pad_amount))

    plt.plot(position.x, position.y, ".", color="#bdbdbd")

    n_timebins = position.n_samples
    rat_position, = ax.plot([], [], "<", color="r")

    fig.tight_layout()


    def init():
        rat_position.set_data([], [])
        return rat_position


    def animate(i):
        rat_position.set_data(position.x[i], position.y[i])
        return rat_position

    anim = animation.FuncAnimation(fig, animate, frames=n_timebins, interval=80, 
                                   blit=False, repeat=True)
    return anim

In [ ]:
filename = os.path.join(dataloc, info.position_filename)
# events = nept.load_events(os.path.join(dataloc, info.event_filename), info.event_labels)

In [ ]:
nvt_data = nept.load_nvt(filename)
targets = nvt_data['targets']
times = nvt_data['time']

# Initialize x, y arrays
x = np.zeros(targets.shape)
y = np.zeros(targets.shape)
# time = np.zeros(targets.shape)

In [ ]:
print(np.shape(nvt_data['x']))

In [ ]:
nvt_position = nept.Position(np.hstack(np.array([nvt_data['x'], nvt_data['y']])[..., np.newaxis]), 
                             nvt_data['time'])

In [ ]:
# Check out animation for single trial
anim = animate_trial(nvt_data['x'], nvt_data['y'], nvt_data['time'], start, stop)
HTML(anim.to_html5_video())

In [ ]:
# X and Y are stored in a custom bitfield. See Neuralynx data file format documentation for details.
# Briefly, each record contains up to 50 targets, each stored in 32bit field.
# X field at [20:31] and Y at [4:15].
# TODO: make into a separate function in nept
for target in range(targets.shape[1]):
    this_sample = targets[:, target]
    for sample in range(targets.shape[0]):
        # When the bitfield is equal to zero there is no valid data for that field
        # and remains zero for the rest of the bitfields in the record.
        if this_sample[target] == 0:
            break
        x[sample, target], y[sample, target] = extract_xy(int(this_sample[sample]))

In [ ]:
# Remove columns with no target data
col_idx = (np.sum(x == 0, axis=0) == x.shape[0]) & (np.sum(y == 0, axis=0) == y.shape[0])
x = np.array(x[:, ~col_idx])
y = np.array(y[:, ~col_idx])

In [ ]:
# Remove rows with no target data
row_idx = (np.sum(x == 0, axis=1) == x.shape[1]) | (np.sum(y == 0, axis=1) == y.shape[1])
x = np.array(x[~row_idx])
y = np.array(y[~row_idx])
times = np.array(times[~row_idx])

In [ ]:
x = x / info.scale_targets[0]
y = y / info.scale_targets[1]

In [ ]:
feeder_x_location, feeder_y_location = sort_led_locations(info, events, times)

In [ ]:
# Initialize out_x and out_y as the first target
out_x = np.array(x[:, 0])
out_y = np.array(y[:, 0])

In [ ]:
# Check out animation for single trial
anim = animate_trial(out_x, out_y, times, start, stop)
HTML(anim.to_html5_video())

In [ ]:
# This correction method assumes we are working with two targets
# (eg. subtracts the two targets, averages over two targets, etc.)
if x.shape[1] == 1 and y.shape[1] == 1:
    # Remove jumps to feeder location
    out_x, out_y, times = remove_jumps_to_feeder(out_x, out_y, times, info)

    # Apply a median filter
    out_x, out_y = median_filter(out_x, out_y)

    position = nept.Position(np.hstack(np.array([out_x, out_y])[..., np.newaxis]), times)
    
    1/0
    
elif x.shape[1] == 2 and y.shape[1] == 2:
    # Find indices where only two targets were available
    two_target_idx = (x[:, 1] != 0) | (y[:, 1] != 0)
    subset_indices = [two_target_idx]

elif x.shape[1] == 3 and y.shape[1] == 3:
    # Find indices where all three targets are available
    three_target_idx = (x[:, 2] != 0) | (y[:, 2] != 0)

    # Find indices where only two targets are available
    two_target_idx = ((x[:, 1] != 0) | (y[:, 1] != 0)) & ~three_target_idx
    subset_indices = [two_target_idx, three_target_idx]

else:
    raise NotImplementedError(
        "this number of targets is not handled (%d x targets, %d y targets)" % (x.shape[1], y.shape[1]))

In [ ]:
for subset_idx in subset_indices:
    subset_x, subset_y = correct_targets(x[subset_idx], y[subset_idx],
                                         feeder_x_location[subset_idx], feeder_y_location[subset_idx])
    out_x[subset_idx] = subset_x
    out_y[subset_idx] = subset_y

In [ ]:
# Check out animation for single trial
anim = animate_trial(out_x, out_y, times, start, stop)
HTML(anim.to_html5_video())

In [ ]:
# Remove jumps to feeder location
nojump_x, nojump_y, ttimes = remove_jumps_to_feeder(out_x, out_y, times, info)

In [ ]:
print("total removed:", len(out_x) - len(nojump_x))

In [ ]:
plt.plot(ttimes, nojump_y, "b.", ms=3)
plt.show()

In [ ]:
len(nojump_y)

In [ ]:
# Check out animation for single trial
anim = animate_trial(nojump_x, nojump_y, ttimes, start, stop)
HTML(anim.to_html5_video())

In [ ]:
print("n_samples:", len(out_x))
plt.plot(out_x, out_y, '.')
plt.show()

In [ ]:
print(out_x[:10])

In [ ]:
# Apply a median filter
out_x, out_y = median_filter(nojump_x, nojump_y)

In [ ]:
print(out_x[:10])

In [ ]:
print("n_samples:", len(out_x))
plt.plot(out_x, out_y, '.')
plt.show()

In [ ]:
# Construct a nept.Position object
position = nept.Position(np.hstack(np.array([out_x, out_y])[..., np.newaxis]), ttimes)

In [ ]:
writer = animation.writers['ffmpeg'](fps=18)
dpi = 600
anim.save(os.path.join(output_filepath, "corrected-position-animation.mp4"), writer=writer, dpi=dpi)

plt.close()

In [ ]:
# Check out animation for single trial
anim = animate_trial(position.x, position.y, position.time, start, stop)
HTML(anim.to_html5_video())

In [ ]:
plt.plot(position.time, position.x, "k.", ms=3)
plt.show()

In [ ]:
plt.plot(position.time, position.y, "k.", ms=3)
plt.xlabel("time")
plt.ylabel("y")
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
plt.tight_layout()
plt.savefig(os.path.join(output_filepath, "corrected-position.png"))
# plt.show()

In [ ]:
position.n_samples